https://archive.ics.uci.edu/ml/datasets/iris

In [1]:
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense
from keras.utils import np_utils
import numpy as np
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score


## Base de dados

In [2]:
base = pd.read_csv('iris.csv')

In [3]:
base.head()

,sepal length,sepal width,petal length,petal width,class
0,5.1,3.5,1.4,0.2,Iris-setosa
1,4.9,3.0,1.4,0.2,Iris-setosa
2,4.7,3.2,1.3,0.2,Iris-setosa
3,4.6,3.1,1.5,0.2,Iris-setosa
4,5.0,3.6,1.4,0.2,Iris-setosa


In [4]:
previsores = base.iloc[:,:4].values
classe = base.iloc[:,4].values

### Escalonamento dos previsores

In [5]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
previsores = scaler.fit_transform(previsores)

### Label Encoder

In [9]:
from sklearn.preprocessing import LabelEncoder
labelencoder = LabelEncoder()
classe = labelencoder.fit_transform(classe)
classe_dummy = np_utils.to_categorical(classe)

In [10]:
from sklearn.model_selection import train_test_split
previsores_treinamento, previsores_teste, classe_treinamento, classe_teste = train_test_split(previsores, classe_dummy, test_size=0.25)
previsores_treinamento.shape

(112, 4)

## Estrutura da rede neural

In [ ]:
classificador = Sequential()
classificador.add(Dense(units = 4, activation = 'relu', input_dim = 4))
classificador.add(Dense(units = 4, activation = 'relu'))
classificador.add(Dense(units = 3, activation = 'softmax'))
#softmax é utilizada quando você tem várias categorias como resultado

In [ ]:
classificador.compile(optimizer = 'adam', loss = 'categorical_crossentropy',
                      metrics = ['categorical_accuracy'])
classificador.fit(previsores_treinamento, classe_treinamento, batch_size = 10,
                  epochs = 1000)


In [13]:
resultado = classificador.evaluate(previsores_teste, classe_teste)
previsoes = classificador.predict(previsores_teste)
previsoes = (previsoes > 0.5)


2/2 [==============================] - 0s 2ms/step - loss: 0.5589 - categorical_accuracy: 0.8947


In [14]:
classe_teste2 = [np.argmax(t) for t in classe_teste] #retorna o indice que tem o maior valor
previsoes2 = [np.argmax(t) for t in previsoes]

In [15]:
from sklearn.metrics import confusion_matrix
matriz = confusion_matrix(previsoes2, classe_teste2)

In [16]:
print(matriz)

[[10  0  0]
 [ 0 15  4]
 [ 0  0  9]]


## Validação cruzada

In [32]:
def criarRede():
    classificador = Sequential()
    classificador.add(Dense(units = 4, activation = 'relu', input_dim = 4))
    classificador.add(Dense(units = 4, activation = 'relu'))
    classificador.add(Dense(units = 3, activation = 'softmax'))


    otimizador = keras.optimizers.Adam(learning_rate = 0.001, decay = 0.0001, clipvalue = 0.5)
    classificador.compile(optimizer = otimizador, loss = 'sparse_categorical_crossentropy',
                      metrics = ['accuracy'])


    return classificador

In [33]:
classificador = KerasClassifier(build_fn = criarRede, epochs = 100, batch_size = 10)


resultados = cross_val_score(estimator=classificador, X=previsores, y=classe, cv =10, verbose=0)

/tmp/ipykernel_3841/4164478360.py:1: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  classificador = KerasClassifier(build_fn = criarRede, epochs = 100, batch_size = 10)
/home/hmendes/.local/lib/python3.9/site-packages/sklearn/model_selection/_validation.py:372: FitFailedWarning: 
10 fits failed out of a total of 10.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
10 fits failed with the following error:
Traceback (most recent call last):
  File "/home/hmendes/.local/lib/python3.9/site-packages/sklearn/model_selection/_validation.py", line 681, in _fit_and_score
    estimator.fit(X_train, y_tr